In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel  
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import requests
import base64
from dotenv import load_dotenv


In [6]:
load_dotenv()

True

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
parcer = StrOutputParser()


In [8]:
def get_template_from_image(image_path_or_url):
    """
    Identifies a product from an image using LangChain with Gemini.

    Args:
        image_path_or_url: The local file path or URL of the product image.

    Returns:
        A string containing the identified product information or an error message.
    """
    try:
        # --- 1. Load Image ---
        if image_path_or_url.startswith(("http://", "https://")):
            # Handle URL
            response = requests.get(image_path_or_url)
            response.raise_for_status()
            image_bytes = response.content
        else:
            # Handle local file path
            with open(image_path_or_url, "rb") as image_file:
                image_bytes = image_file.read()

        # --- 2. Prepare the Prompt and Image for LangChain ---
        prompt_text = """
        Analyze the provided image of a product and return the following information in a structured manner with clear labels without bold or italics. The information to extract includes:
        1.  Product Name: The primary name of the product.
        2.  Product Type: The category of the product (e.g., Food, Cosmetic, Cleaning Supply).
        3.  Extracted Information: All ingredients and chemicals, and warnings and certifications.
        4.  Brand: The brand name of the product.
        Provide the response in a clear and concise format. If any information is not available, indicate it as "Not Available". Do not include any additional commentary or explanations.
        """
        
        image_b64 = base64.b64encode(image_bytes).decode("utf-8")

        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{image_b64}",
                },
            ]
        )
        return message
    except Exception as e:
        return f"Error loading image: {str(e)}"
    
    

In [ ]:
h_msg = get_template_from_image("Kurkure.jpg")

# Use the prepared image HumanMessage (h_msg) with the Gemini llm


In [ ]:
chat_history = [
    SystemMessage(content=(
        """You are an expert dietitian and nutritionist. Provide clear, evidence-based guidance on what foods and nutrients to consume for overall health, recommended portions, and which foods or ingredients are harmful or should be avoided. Prioritize safety, note common contraindications and allergies, and advise when to consult a healthcare professional. Be concise and practical."""
    )),
    h_msg
]

In [ ]:
result = llm.invoke(chat_history)

In [12]:
product_info = parcer.invoke(result)
print("Identified Product Information:\n", product_info)

Identified Product Information:
 Product Name: Kurkure
Product Type: Food
Extracted Information:
Ingredients: Cereal Products (57%) (Rice Meal (38%), Corn Meal (19%)), Edible Vegetable Oil (Palmolein), Seasoning (Spices & Condiments, Sugar, Iodised Salt, Acidity Regulators (330, 296), Flavour (Natural and Nature Identical Flavouring Substances), Flavour Enhancers (627, 631), Edible Vegetable Oil (Corn Oil), Color (160c), Emulsifier (414)), Gram Meal (6%).
Chemicals: Acidity Regulators (330, 296), Flavour Enhancers (627, 631), Color (160c), Emulsifier (414).
Warnings: Contains Onion and Garlic (as flavouring agent).
Certifications: Fssai, Mkt. Lic. No. 10014064000435, Vegetarian symbol (green circle with a dot).
Brand: Kurkure


In [14]:
# from langchain_core.prompts import ChatPromptTemplate

text = f'''Analyze the product below and return a concise, structured report with clear labels (no bold or italics). Use {product_info} as the source. Include:
- Product Summary: one-line description.
- Pros: short bullet list of benefits.
- Cons: short bullet list of risks/harmful aspects.
- Nutritional & Safety Considerations: key points (fat, sodium, sugar, allergens, additives) and any special cautions.
- Who Should Limit/Avoid: specific populations.
- Recommended Serving/Frequency: practical guidance.
- Regulatory Status: allowed/banned status for product and key additives in India, EU, USA, Canada, Australia (or "Not Available" if unknown).
- Healthier Alternatives or Tips: brief suggestions.

Do not add extra commentary; be concise and user-focused.'''

chat_history = [
    SystemMessage(content='You are an expert in human health and dietary supplements.'),
    HumanMessage(content=f'{text}')
]

result = llm.invoke(chat_history)
chat_history.append(AIMessage(content=result.content))
print("\n--- Detailed Product Information ---")
print(result.content)




--- Detailed Product Information ---
Product Name: Kurkure
Product Type: Food

Product Summary: A processed snack food primarily made from rice and corn meal, flavored with spices, salt, sugar, and various food additives.

Pros:
*   Provides immediate caloric energy.
*   Convenient and widely available snack option.
*   Vegetarian friendly as indicated by the symbol.

Cons:
*   High in processed ingredients and refined cereals.
*   Contains multiple food additives, including flavor enhancers and artificial colors.
*   Likely high in fat, sodium, and sugar, typical of such snack foods.

Nutritional & Safety Considerations:
*   Fat: Contains edible vegetable oils (Palmolein, Corn Oil) which contribute to fat content. Palmolein can be high in saturated fat.
*   Sodium: Iodised Salt is a key ingredient in the seasoning.
*   Sugar: Sugar is listed in the seasoning mix.
*   Allergens: Contains Onion and Garlic as flavoring agents. No other major allergens explicitly listed, but cross-contam

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()

